# HIV Model
Austin Veseliza

In [4]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
import matplotlib.pyplot as plt


# import functions from the modsim.py module
from modsim import *

In [5]:
init_state = State(R=1000, L=0, E=0, V=0.0000004)

,values
R,1.000000e+03
L,0.000000e+00
E,0.000000e+00
V,4.000000e-07


In [6]:
system = System(
    CD4_arise_rate  = 1.36, 
    CD4_active_prop = 0.2,
    CD4_death_rate    = 1.36 * 10**(-3),
    R_infect_rate   = 0.00027,
    
    latent_infect_prop = 0.1,
    L_activate_rate    = 3.6 * 10**(-2),
    E_death_rate       = 0.33,
    
    V_arise_rate = 100,
    V_death_rate = 2
)

,values
CD4_arise_rate,1.36000
CD4_active_prop,0.20000
CD4_death_rate,0.00136
R_infect_rate,0.00027
latent_infect_prop,0.10000
L_activate_rate,0.03600
E_death_rate,0.33000
V_arise_rate,100.00000
V_death_rate,2.00000


In [7]:
def R_in(system, state):
    return system.CD4_arise_rate * system.CD4_active_prop

def R_out(system, state):
    return (system.CD4_death_rate * state.R) + (system.R_infect_rate * state.R * state.V)

def R_delta(system, state):
    return R_in(system, state) - R_out(system, state)

In [8]:
def L_in(system, state):
    return system.latent_infect_prop * system.R_infect_rate * state.R * state.V

def L_out(system, state):
    return (system.CD4_death_rate * state.L) + (system.L_activate_rate * state.L)

def L_delta(system, state):
    return L_in(system, state) - L_out(system, state)

In [9]:
def E_in(system, state):
    return ((1 - system.latent_infect_prop) * system.R_infect_rate * state.R * state.V) + (system.L_activate_rate * state.L)

def E_out(system, state):
    return system.E_death_rate * state.E

def E_delta(system, state):
    return E_in(system, state) - E_out(system, state)

In [10]:
def V_in(system, state):
    return system.V_arise_rate * state.E

def V_out(system, state):
    return system.V_death_rate * state.V

def V_delta(system, state):
    return V_in(system, state) - V_out(system, state)

In [11]:
def step(system, state):
    r = state.R + R_delta(system, state)
    l = state.L + L_delta(system, state)
    e = state.E + E_delta(system, state)
    v = state.V + V_delta(system, state)
    
    return State(R=r, L=l, E=e, V=v)

In [14]:
def run_simulation(system, init_state, num_steps):
    
    results = TimeFrame(columns=init_state.index)
    
    results.row[0] = init_state
    
    for t in range(num_steps):
        results.row[t+1] = step(system, results.row[t])
    
    return results

In [15]:
results = run_simulation(system, init_state, 10)

,R,L,E,V
0,1000.000000,0.000000e+00,0.000000e+00,4.000000e-07
1,998.912000,1.080000e-08,9.720000e-08,-4.000000e-07
2,997.825480,-3.917376e-10,-3.158145e-08,1.012000e-05
3,996.740434,2.722687e-07,2.432639e-06,-1.327814e-05
4,995.656871,-9.524455e-08,-1.576402e-06,2.565420e-04
5,994.574709,6.804865e-06,6.100935e-05,-4.141822e-04
6,993.494198,-4.571615e-06,-5.897901e-05,6.515117e-03
7,992.413298,1.703629e-04,1.533193e-03,-1.241302e-02
8,991.338942,-1.686106e-04,-1.960107e-03,1.657323e-01
9,990.218361,4.273705e-03,3.860481e-02,-3.617430e-01
